# Import Packages and Initiate Dataframes

In [1]:
cd ..

/home/jovyan/Project_3_madelon


In [2]:
%matplotlib inline

In [3]:
%run __init__.py

In [4]:
features = [48, 64, 105, 128, 241, 336, 338, 378, 442, 453, 475]
informative_features = [28, 48, 64, 105, 128, 153, 241, 281, 318, 336,
                        338, 378, 433, 442, 451, 453, 455, 472, 475, 493]

In [5]:
madelon_train_full = madelon_train
train_targets = madelon_train_targets
train_df = madelon_train_full[informative_features]
train_df['targets'] = train_targets

In [6]:
madelon_val_full = madelon_valid
val_targets = madelon_val_target
val_df = madelon_val_full[informative_features]
val_df['targets'] = val_targets

# KNeighborsClassifier Pipeline with 20 features

In [7]:
# creates a KNeighborsClassifier pipeline with SelectKBest and StandardScaler instances (20 features)

knc_pipeline = Pipeline([
    ('scaler',StandardScaler()),
    ('skb', SelectKBest()),
    ('knc', KNeighborsClassifier())
])

In [8]:
# checks parameters and returns the best parameters for each one listed below. Note: some parameters are commented out 
# for compiling time reasons. I did run all of these commented out codes at least one time through and the optimal 
# parameters are used in the function that follows where only 10% of the dataset is used.

knc_params = {
    #'knc__n_neighbors': np.arange(1, 30, 1),
    'knc__weights': ['uniform', 'distance'],
    #'knc__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    #'skb__k': np.arange(1, 11, 1)
}

In [9]:
# creates a GridSearchCV instance to search for the optimal parameters for this model

knc_gs = GridSearchCV(knc_pipeline, 
                      knc_params,
                      cv=5,
                      n_jobs = -1)

In [10]:
# fits model using the train_df (20 features) and the train_targets

knc_gs.fit(X=train_df, y=train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('skb', SelectKBest(k=10, score_func=<function f_classif at 0x7f1591ab4510>)), ('knc', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'knc__weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [11]:
# gets best parameters (used in the function call below)

knc_gs.best_params_

{'knc__weights': 'uniform'}

In [12]:
# computes the train score and test score for both raw and scaled data for this particular model using only 10% of the dataset
# 200 train observations, 60 validation observations sampled 3 separate times

knc = add_train_test_score_df(train_df, val_df, KNeighborsClassifier(algorithm = 'auto', 
                                                                     n_neighbors = 6, 
                                                                     weights = 'distance'), feats = 20)
knc_scaled = add_train_test_score_df(train_df, val_df, KNeighborsClassifier(algorithm = 'auto', 
                                                                     n_neighbors = 6, 
                                                                     weights = 'distance'), feats = 20, scaled = True)
knc, knc_scaled

([1.0, 0.75], [1.0, 0.79999999999999993])

# KNeighborsClassifier Pipeline with 5 features

In [13]:
# creates a KNeighborsClassifier pipeline with SelectKBest and StandardScaler instances (5 features)

knc_pipeline_s5b = Pipeline([
    ('scaler',StandardScaler()),
    ('knc', KNeighborsClassifier())
])

In [14]:
# checks parameters and returns the best parameters for each one listed below. Note: some parameters are commented out 
# for compiling time reasons. I did run all of these commented out codes at least one time through and the optimal 
# parameters are used in the function that follows where only 10% of the dataset is used.

knc_params_s5b = {
    #'knc__n_neighbors': np.arange(1, 30, 1),
    'knc__weights': ['uniform', 'distance'],
    #'knc__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
}

In [15]:
# creates a GridSearchCV instance to search for the optimal parameters for this model

knc_gs_s5b = GridSearchCV(knc_pipeline_s5b, 
                      knc_params_s5b,
                      cv=5,
                      n_jobs = -1)

In [16]:
# uses only the SKBest = 5 features from the feature selection notebook to train the model

skb_feats = [64, 128, 241, 336, 475]
train_skb_df = train_df[skb_feats]

val_skb_df = val_df[skb_feats]

In [17]:
# fits model using the train_df (SKBest = 5 features) and the train_targets

knc_gs_s5b.fit(X=train_skb_df, y=train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knc', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'knc__weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [18]:
# gets best parameters (used in the function call below)

knc_gs_s5b.best_params_

{'knc__weights': 'uniform'}

In [19]:
# returns train score for entire dataset (2000 observations) using SKBest = 5 features

knc_gs_s5b.score(train_skb_df, train_targets[0])

0.8165

In [20]:
# returns validation score for entire dataset (600 observations) using SKBest = 5 features

knc_gs_s5b.score(val_skb_df, val_targets[0])

0.71666666666666667

In [21]:
# adds the targets to the data set so it is in the correct format for the created function

train_skb_df['targets'] = train_targets
val_skb_df['targets'] = val_targets

In [22]:
# computes the train score and test score for both raw and scaled data for this particular model using only 10% of the dataset samples 3 times.
# 200 train observations, 60 validation observations sampled 3 separate times

knc_skb = add_train_test_score_df(train_skb_df, val_skb_df, KNeighborsClassifier(algorithm = 'auto', 
                                                                     n_neighbors = 17, 
                                                                     weights = 'distance'), feats = 5)
knc_skb_scaled = add_train_test_score_df(train_skb_df, val_skb_df, KNeighborsClassifier(algorithm = 'auto', 
                                                                     n_neighbors = 17, 
                                                                     weights = 'distance'), feats = 5, scaled = True)
knc_skb, knc_skb_scaled

([1.0, 0.58333333333333337], [1.0, 0.58888888888888891])

# DecisionTreeClassifier Pipeline with 5 features (SKBest)

In [23]:
# creates a DecisionTreeClassifier pipeline with SelectKBest and StandardScaler instances (5 features)

dtc_pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', DecisionTreeClassifier(random_state=42))
])

In [24]:
# checks parameters and returns the best parameters for each one listed below. Note: some parameters are commented out 
# for compiling time reasons. I did run all of these commented out codes at least one time through and the optimal 
# parameters are used in the function that follows where only 10% of the dataset is used.

dtc_params = {
    'clf__max_depth': [1,2,3,4,5,6,7,8,None],
    'clf__min_samples_split':np.arange(0.01,1,15),
    'clf__criterion':['gini','entropy'],
    'clf__max_features':np.arange(.01, 1, 15),
    
}

In [25]:
# creates a GridSearchCV instance to search for the optimal parameters for this model

dtc_gs = GridSearchCV(dtc_pipe, 
                      dtc_params,
                      cv=5,
                      n_jobs = -1)

In [26]:
# uses only the SKBest = 5 features from the feature selection notebook to train the model

skb_feats = [64, 128, 241, 336, 475]
train_skb_df = train_df[skb_feats]

val_skb_df = val_df[skb_feats]

In [27]:
# fits model using the train_df (SKBest = 5 features) and the train_targets

dtc_gs.fit(train_skb_df, train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, None], 'clf__min_samples_split': array([ 0.01]), 'clf__criterion': ['gini', 'entropy'], 'clf__max_features': array([ 0.01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [28]:
# gets best parameters (used in the function call below)

dtc_gs.best_params_

{'clf__criterion': 'entropy',
 'clf__max_depth': None,
 'clf__max_features': 0.01,
 'clf__min_samples_split': 0.01}

In [29]:
# returns train score for entire dataset (2000 observations) using SKBest = 5 features

dtc_gs.score(train_skb_df, train_targets[0])

0.80649999999999999

In [30]:
# returns validation score for entire dataset (600 observations) using SKBest = 5 features

dtc_gs.score(val_skb_df, val_targets[0])

0.64166666666666672

In [31]:
# adds the targets to the data set so it is in the correct format for the created function

train_skb_df['targets'] = train_targets
val_skb_df['targets'] = val_targets

In [45]:
# computes the train score and test score for both raw and scaled data for this particular model using only 10% of the dataset samples 3 times.
# 200 train observations, 60 validation observations sampled 3 separate times

dtc_skb = add_train_test_score_df(train_skb_df, val_skb_df, DecisionTreeClassifier(criterion = 'entropy',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), feats = 5)
dtc_skb_scaled = add_train_test_score_df(train_skb_df, val_skb_df, DecisionTreeClassifier(criterion = 'entropy',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), feats = 5,
                                                                                     scaled = True)
dtc_skb, dtc_skb_scaled

([1.0, 0.58333333333333337], [1.0, 0.5722222222222223])

# DecisionTreeClassifier Pipeline with 5 features (RFE)

In [46]:
# creates a DecisionTreeClassifier pipeline with RFE and StandardScaler instances (5 features)


dtc_pipe_rfe = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', DecisionTreeClassifier(random_state=42))
])

In [47]:
# checks parameters and returns the best parameters for each one listed below. Note: some parameters are commented out 
# for compiling time reasons. I did run all of these commented out codes at least one time through and the optimal 
# parameters are used in the function that follows where only 10% of the dataset is used.

dtc_params_rfe = {
    'clf__max_depth': [1,2,3,4,5,6,7,8,None],
    'clf__min_samples_split':np.arange(0.01,1,15),
    'clf__criterion':['gini','entropy'],
    'clf__max_features':np.arange(.01, 1, 15),
    
}

In [48]:
# creates a GridSearchCV instance to search for the optimal parameters for this model

dtc_gs_rfe = GridSearchCV(dtc_pipe_rfe, 
                      dtc_params_rfe,
                      cv=5,
                      n_jobs = -1)

In [50]:
# uses only the top 5 RFE features from the feature selection notebook to train the model

rfe_feats = [48, 105, 338, 442, 475]

train_rfe_df = train_df[rfe_feats]
val_rfe_df = val_df[rfe_feats]

In [51]:
# fits model using the train_df (top 5 RFE features) and the train_targets

dtc_gs_rfe.fit(train_rfe_df, train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, None], 'clf__min_samples_split': array([ 0.01]), 'clf__criterion': ['gini', 'entropy'], 'clf__max_features': array([ 0.01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [52]:
# gets best parameters (used in the function call below)

dtc_gs_rfe.best_params_

{'clf__criterion': 'gini',
 'clf__max_depth': None,
 'clf__max_features': 0.01,
 'clf__min_samples_split': 0.01}

In [53]:
# returns train score for entire dataset (2000 observations) using top 5 RFE features

dtc_gs_rfe.score(train_rfe_df, train_targets[0])

0.85699999999999998

In [54]:
# returns validation score for entire dataset (600 observations) using top 5 RFE features

dtc_gs_rfe.score(val_rfe_df, val_targets[0])

0.71666666666666667

In [55]:
# adds the targets to the data set so it is in the correct format for the created function

train_rfe_df['targets'] = train_targets
val_rfe_df['targets'] = val_targets

In [56]:
# computes the train score and test score for both raw and scaled data for this particular model using only 10% of the dataset samples 3 times.
# 200 train observations, 60 validation observations sampled 3 separate times

dtc_rfe = add_train_test_score_df(train_rfe_df, val_rfe_df, DecisionTreeClassifier(criterion = 'gini',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), feats = 5)
dtc_rfe_scaled = add_train_test_score_df(train_rfe_df, val_rfe_df, DecisionTreeClassifier(criterion = 'gini',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), feats = 5,
                                                                                     scaled = True)
dtc_rfe, dtc_rfe_scaled

([1.0, 0.53333333333333333], [1.0, 0.53888888888888886])

# DecisionTreeClassifier Pipeline with 10 features (SFM)

In [57]:
# creates a DecisionTreeClassifier pipeline with SFM and StandardScaler instances (10 features)

dtc_pipe_sfm = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', DecisionTreeClassifier(random_state=42))
])

In [58]:
# checks parameters and returns the best parameters for each one listed below. Note: some parameters are commented out 
# for compiling time reasons. I did run all of these commented out codes at least one time through and the optimal 
# parameters are used in the function that follows where only 10% of the dataset is used.

dtc_params_sfm = {
    'clf__max_depth': [1,2,3,4,5,6,7,8,None],
    'clf__min_samples_split':np.arange(0.01,1,15),
    'clf__criterion':['gini','entropy'],
    'clf__max_features':np.arange(.01, 1, 15),
    
}

In [59]:
# creates a GridSearchCV instance to search for the optimal parameters for this model

dtc_gs_sfm = GridSearchCV(dtc_pipe_sfm, 
                      dtc_params_sfm,
                      cv=5,
                      n_jobs = -1)

In [60]:
# uses only the top 10 SFM features from the feature selection notebook to train the model

sfm_feats = [28, 48, 105, 153, 338, 378, 442, 451, 453, 475]

train_sfm_df = train_df[sfm_feats]
val_sfm_df = val_df[sfm_feats]

In [61]:
# fits model using the train_df (top 10 SFM features) and the train_targets

dtc_gs_sfm.fit(train_sfm_df, train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best'))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, None], 'clf__min_samples_split': array([ 0.01]), 'clf__criterion': ['gini', 'entropy'], 'clf__max_features': array([ 0.01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [62]:
# gets best parameters (used in the function call below)

dtc_gs_sfm.best_params_

{'clf__criterion': 'gini',
 'clf__max_depth': None,
 'clf__max_features': 0.01,
 'clf__min_samples_split': 0.01}

In [63]:
# returns train score for entire dataset (2000 observations) using top 10 SFM features

dtc_gs_sfm.score(train_sfm_df, train_targets[0])

0.86450000000000005

In [64]:
# returns validation score for entire dataset (600 observations) using top 10 SFM features

dtc_gs_sfm.score(val_sfm_df, val_targets[0])

0.75166666666666671

In [65]:
# adds the targets to the data set so it is in the correct format for the created function

train_sfm_df['targets'] = train_targets
val_sfm_df['targets'] = val_targets

In [66]:
# computes the train score and test score for both raw and scaled data for this particular model using only 10% of the dataset samples 3 times.
# 200 train observations, 60 validation observations sampled 3 separate times

dtc_sfm = add_train_test_score_df(train_sfm_df, val_sfm_df, DecisionTreeClassifier(criterion = 'gini',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), feats = 10)
dtc_sfm_scaled = add_train_test_score_df(train_sfm_df, val_sfm_df, DecisionTreeClassifier(criterion = 'gini',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), feats = 10,
                                                                                     scaled = True)
dtc_sfm, dtc_sfm_scaled

([1.0, 0.57222222222222219], [1.0, 0.62777777777777777])

# DecisionTreeClassifier with BaggingClassifier using SKBest = 5

In [68]:
# creates a DecisionTreeClassifier with a BaggingClassifier pipeline with SKBest and StandardScaler instances (5 features)

bag_dtc_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', BaggingClassifier(DecisionTreeClassifier(random_state=42), 
                              max_samples=.8, 
                              random_state=42))
])

In [69]:
# gets a list of all possible parameters that can be tested using GridSearchCV

bag_dtc_pipe.get_params().keys()

dict_keys(['steps', 'scaler', 'clf', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'clf__base_estimator__class_weight', 'clf__base_estimator__criterion', 'clf__base_estimator__max_depth', 'clf__base_estimator__max_features', 'clf__base_estimator__max_leaf_nodes', 'clf__base_estimator__min_impurity_split', 'clf__base_estimator__min_samples_leaf', 'clf__base_estimator__min_samples_split', 'clf__base_estimator__min_weight_fraction_leaf', 'clf__base_estimator__presort', 'clf__base_estimator__random_state', 'clf__base_estimator__splitter', 'clf__base_estimator', 'clf__bootstrap', 'clf__bootstrap_features', 'clf__max_features', 'clf__max_samples', 'clf__n_estimators', 'clf__n_jobs', 'clf__oob_score', 'clf__random_state', 'clf__verbose', 'clf__warm_start'])

In [70]:
# checks parameters and returns the best parameters for each one listed below. Note: some parameters are commented out 
# for compiling time reasons. I did run all of these commented out codes at least one time through and the optimal 
# parameters are used in the function that follows where only 10% of the dataset is used.

bag_dtc_params = {
    'clf__base_estimator__criterion': ['gini', 'entropy'],
    #'clf__base_estimator__max_depth': [1,2,3,4,5,6,7,8,None],
    #'clf__base_estimator__max_features': np.arange(.01, 1, 15),
    #'clf__base_estimator__min_samples_split': np.arange(0.01,1,15),
    #'clf__max_features':np.arange(1, 6, 1),
    #'clf__n_estimators':np.arange(5, 50, 5)
    #[DecisionTreeClassifier(max_depth=md, criterion = ['gini', 'entropy'],
                                                  # min_samples_split=np.arange(0.01,1,15),
                                                  # max_features=np.arange(0.01, 1, 15),
                                                  # random_state=42)
                            # for md in [5,7,10,None]],
}

In [71]:
# creates a GridSearchCV instance to search for the optimal parameters for this model

bag_dtc_gs = GridSearchCV(bag_dtc_pipe, bag_dtc_params, cv=5, n_jobs=-1)

In [72]:
# uses only the top 5 SKBest features from the feature selection notebook to train the model

skb_feats = [64, 128, 241, 336, 475]
train_skb_df = train_df[skb_feats]

val_skb_df = val_df[skb_feats]

In [73]:
# fits model using the train_df (top 10 SFM features) and the train_targets

bag_dtc_gs.fit(train_skb_df, train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf...n_estimators=10, n_jobs=1, oob_score=False,
         random_state=42, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__base_estimator__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [74]:
# gets best parameters (used in the function call below)

bag_dtc_gs.best_params_

{'clf__base_estimator__criterion': 'entropy'}

In [75]:
# returns train score for entire dataset (2000 observations) using top 5 SKBest features and BaggingClassifier

bag_dtc_gs.score(train_skb_df, train_targets[0])

0.95750000000000002

In [76]:
# returns validation score for entire dataset (600 observations) using top 5 SKBest features and BaggingClassifier

bag_dtc_gs.score(val_skb_df, val_targets[0])

0.70999999999999996

In [77]:
# adds the targets to the data set so it is in the correct format for the created function

train_skb_df['targets'] = train_targets
val_skb_df['targets'] = val_targets

In [78]:
# computes the train score and test score for both raw and scaled data for this particular model (with BaggingClassifier) using only 10% of the dataset samples 3 times.
# 200 train observations, 60 validation observations sampled 3 separate times

bag_dtc_skb = add_train_test_score_df(train_skb_df, val_skb_df, BaggingClassifier(DecisionTreeClassifier(criterion = 'entropy',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), 
                                                                                n_estimators = 15,
                                                                                  max_features = 5,
                                                                                  random_state = 42), feats = 5)
bag_dtc_skb_scaled = add_train_test_score_df(train_skb_df, val_skb_df, BaggingClassifier(DecisionTreeClassifier(criterion = 'entropy',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), 
                                                                                n_estimators = 15,
                                                                                         max_features = 5,
                                                                                        random_state = 42), feats = 5,
                                                                                     scaled = True)
bag_dtc_skb, bag_dtc_skb_scaled

([0.99333333333333329, 0.61111111111111116],
 [0.99333333333333329, 0.62777777777777777])

# DecisionTreeClassifier with BaggingClassifier using RFE

In [32]:
# creates a DecisionTreeClassifier with a BaggingClassifier pipeline with RFE and StandardScaler instances (5 features)

bag_dtc_rfe_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', BaggingClassifier(DecisionTreeClassifier(random_state=42), 
                              max_samples=.8, 
                              random_state=42))
])

In [33]:
# checks parameters and returns the best parameters for each one listed below. Note: some parameters are commented out 
# for compiling time reasons. I did run all of these commented out codes at least one time through and the optimal 
# parameters are used in the function that follows where only 10% of the dataset is used.

bag_dtc_rfe_params = {
    'clf__base_estimator__criterion': ['gini', 'entropy'],
    #'clf__base_estimator__max_depth': [1,2,3,4,5,6,7,8,None],
    #'clf__base_estimator__max_features': np.arange(.01, 1, 15),
    #'clf__base_estimator__min_samples_split': np.arange(0.01,1,15),
    #'clf__max_features':np.arange(1, 6, 1),
    #'clf__n_estimators':np.arange(5, 50, 5)
    #[DecisionTreeClassifier(max_depth=md, criterion = ['gini', 'entropy'],
                                                  # min_samples_split=np.arange(0.01,1,15),
                                                  # max_features=np.arange(0.01, 1, 15),
                                                  # random_state=42)
                            # for md in [5,7,10,None]],
}

In [34]:
# creates a GridSearchCV instance to search for the optimal parameters for this model

bag_dtc_rfe_gs = GridSearchCV(bag_dtc_rfe_pipe, bag_dtc_rfe_params, cv=5, n_jobs=-1)

In [35]:
# uses only the top 5 RFE features from the feature selection notebook to train the model

rfe_feats = [48, 105, 338, 442, 475]

train_rfe_df = train_df[rfe_feats]
val_rfe_df = val_df[rfe_feats]

In [36]:
# fits model using the train_df (top 5 RFE features) and the train_targets

bag_dtc_rfe_gs.fit(train_rfe_df, train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf...n_estimators=10, n_jobs=1, oob_score=False,
         random_state=42, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__base_estimator__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [37]:
# gets best parameters (used in the function call below)

bag_dtc_rfe_gs.best_params_

{'clf__base_estimator__criterion': 'gini'}

In [38]:
# returns train score for entire dataset (2000 observations) using top 5 RFE features and BaggingClassifier

bag_dtc_rfe_gs.score(train_rfe_df, train_targets[0])

0.97499999999999998

In [39]:
# returns validation score for entire dataset (600 observations) using top 5 RFE features and BaggingClassifier

bag_dtc_rfe_gs.score(val_rfe_df, val_targets[0])

0.84166666666666667

In [40]:
# adds the targets to the data set so it is in the correct format for the created function

train_rfe_df['targets'] = train_targets
val_rfe_df['targets'] = val_targets

In [41]:
# computes the train score and test score for both raw and scaled data for this particular model (with BaggingClassifier) using only 10% of the dataset samples 3 times.
# 200 train observations, 60 validation observations sampled 3 separate times

bag_dtc_rfe = add_train_test_score_df(train_rfe_df, val_rfe_df, BaggingClassifier(DecisionTreeClassifier(criterion = 'entropy',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), 
                                                                                n_estimators = 40,
                                                                                  max_features = 5,
                                                                                  random_state = 42), feats = 5)
bag_dtc_rfe_scaled = add_train_test_score_df(train_rfe_df, val_rfe_df, BaggingClassifier(DecisionTreeClassifier(criterion = 'entropy',
                                                                                  max_depth = None,
                                                                                  max_features = 0.01,
                                                                                  min_samples_split = 0.01,
                                                                                  random_state = 42), 
                                                                                n_estimators = 40,
                                                                                         max_features = 5,
                                                                                        random_state = 42), feats = 5,
                                                                                     scaled = True)
bag_dtc_rfe, bag_dtc_rfe_scaled

([1.0, 0.7055555555555556], [1.0, 0.71111111111111114])

# RandomForestClassifier using 5 features (RFE)

In [42]:
# creates a RandomForestClassifier pipeline with RFE and StandardScaler instances (5 features)
# checks parameters and returns the best parameters for each one listed below. Note: some parameters are commented out 
# for compiling time reasons. I did run all of these commented out codes at least one time through and the optimal 
# parameters are used in the function that follows where only 10% of the dataset is used.

rf_pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('clf',RandomForestClassifier(random_state=42))
])

rfparams = {
    'clf__criterion':['gini', 'entropy'],
    #'clf__max_depth':[1, 2, 3, 4, 5, 6, 7, 8, None],
    #'clf__n_estimators':[10,50,100,200],
    #'clf__max_features':['auto','log2']
}

In [43]:
# creates a GridSearchCV instance to search for the optimal parameters for this model

rfgs = GridSearchCV(rf_pipe, rfparams, cv=5, n_jobs=-1)

In [44]:
# uses only the top 5 RFE features from the feature selection notebook to train the model

rf_rfe_feats = [105, 241, 318, 338, 378]
train_rf_rfe = train_df[rf_rfe_feats]
val_rf_rfe = val_df[rf_rfe_feats]

In [45]:
# fits model using the train_df (top 5 RFE features) and the train_targets

rfgs.fit(train_rf_rfe, train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [46]:
# gets best parameters (used in the function call below)

rfgs.best_params_

{'clf__criterion': 'entropy'}

In [47]:
# returns train score for entire dataset (2000 observations) using top 5 RFE features

rfgs.score(train_rf_rfe,train_targets[0])

0.99150000000000005

In [49]:
# returns validation score for entire dataset (600 observations) using top 5 RFE features

rfgs.score(val_rf_rfe,val_targets[0])

0.84333333333333338

In [50]:
# adds the targets to the data set so it is in the correct format for the created function

train_rf_rfe['targets'] = train_targets
val_rf_rfe['targets'] = val_targets

In [51]:
# computes the train score and test score for both raw and scaled data for this particular model using only 10% of the dataset samples 3 times.
# 200 train observations, 60 validation observations sampled 3 separate times

rf_rfe = add_train_test_score_df(train_rf_rfe, val_rf_rfe, RandomForestClassifier(criterion = 'entropy',
                                                                                  max_depth = None,
                                                                                  max_features = 'auto',
                                                                                  n_estimators = 100,
                                                                                random_state = 42), feats = 5)
rf_rfe_scaled = add_train_test_score_df(train_rf_rfe, val_rf_rfe, RandomForestClassifier(criterion = 'entropy',
                                                                                    max_depth = None,
                                                                                    max_features = 'auto',
                                                                                    n_estimators = 100,
                                                                                    random_state = 42), feats = 5,
                                                                                    scaled = True)
rf_rfe, rf_rfe_scaled

([1.0, 0.67222222222222217], [1.0, 0.65555555555555545])

# Bagging using RandomForestClassifier with 5 features (RFE)

In [52]:
# creates a RandomForestClassifier with BaggingClassifier pipeline with RFE and StandardScaler instances (5 features)
# checks parameters and returns the best parameters for each one listed below. Note: some parameters are commented out 
# for compiling time reasons. I did run all of these commented out codes at least one time through and the optimal 
# parameters are used in the function that follows where only 10% of the dataset is used.

bag_rf_pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('clf',BaggingClassifier(RandomForestClassifier(criterion = 'entropy',
                                                    max_depth = None,
                                                    max_features = 'auto',
                                                    n_estimators = 100,
                                                    random_state = 42), max_samples = .8, random_state=42))
])

bag_rfparams = {
    #'clf__base_estimator__criterion':['gini', 'entropy'],
    #'clf__base_estimator__max_depth':[1, 2, 3, 4, 5, 6, 7, 8, None],
    #'clf__base_estimator__n_estimators':[10,50,100,200],
    #'clf__base_estimator__max_features':['auto','log2'],
    'clf__max_features':np.arange(1, 6, 1),
    #'clf__n_estimators':np.arange(5, 50, 5)
}

In [54]:
# creates a GridSearchCV instance to search for the optimal parameters for this model

bag_rfgs = GridSearchCV(bag_rf_pipe, bag_rfparams, cv=5, n_jobs=-1)

In [55]:
# uses only the top 5 RFE features from the feature selection notebook to train the model

rf_rfe_feats = [105, 241, 318, 338, 378]
train_rf_rfe = train_df[rf_rfe_feats]
val_rf_rfe = val_df[rf_rfe_feats]

In [56]:
# fits model using the train_df (top 5 RFE features) and the train_targets

bag_rfgs.fit(train_rf_rfe, train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e...n_estimators=10, n_jobs=1, oob_score=False,
         random_state=42, verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__max_features': array([1, 2, 3, 4, 5])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [57]:
# gets best parameters (used in the function call below)

bag_rfgs.best_params_

{'clf__max_features': 5}

In [58]:
# returns train score for entire dataset (2000 observations) using top 5 RFE features and BaggingClassifier

bag_rfgs.score(train_rf_rfe,train_targets[0])

0.97150000000000003

In [59]:
# returns validation score for entire dataset (600 observations) using top 5 RFE features and BaggingClassifier

bag_rfgs.score(val_rf_rfe,val_targets[0])

0.85333333333333339

In [60]:
# adds the targets to the data set so it is in the correct format for the created function

train_rf_rfe['targets'] = train_targets
val_rf_rfe['targets'] = val_targets

In [61]:
# computes the train score and test score for both raw and scaled data for this particular model (with BaggingClassifier) using only 10% of the dataset samples 3 times.
# 200 train observations, 60 validation observations sampled 3 separate times

bag_rf_rfe = add_train_test_score_df(train_rf_rfe, val_rf_rfe, BaggingClassifier(RandomForestClassifier(criterion = 'entropy',
                                                                                  max_depth = None,
                                                                                  max_features = 'auto',
                                                                                  n_estimators = 100,
                                                                                random_state = 42), 
                                                                                max_features = 5,
                                                                                n_estimators = 45,
                                                                                 max_samples = 0.8,
                                                                                random_state = 42
                                                                                ), feats = 5)
bag_rf_rfe_scaled = add_train_test_score_df(train_rf_rfe, val_rf_rfe, BaggingClassifier(RandomForestClassifier(criterion = 'entropy',
                                                                                    max_depth = None,
                                                                                    max_features = 'auto',
                                                                                    n_estimators = 100,
                                                                                    random_state = 42),
                                                                                    max_features = 5,
                                                                                    n_estimators = 45,
                                                                                        max_samples = 0.8,
                                                                                    random_state = 42
                                                                                       ), feats = 5,
                                                                                    scaled = True)
bag_rf_rfe, bag_rf_rfe_scaled

([0.97999999999999998, 0.66666666666666663],
 [0.97999999999999998, 0.65555555555555556])

# RandomForestClassifier with 11 features (after Further Exploration section in Feature Selection notebook)

In [91]:
eleven_feats = [48, 64, 105, 128, 241, 336, 338, 378, 442, 453, 475]

In [119]:
eleven_val_df = madelon_valid[eleven_feats]

In [120]:
eleven_train_df = madelon_train[eleven_feats]

In [103]:

rf_pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('clf',RandomForestClassifier(random_state=42, n_estimators = 350))
])

rfparams = {
    'clf__max_features':['auto','log2'],
    'clf__criterion':['gini', 'entropy'],
    'clf__max_depth':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, None]
}

In [104]:
rfgs = GridSearchCV(rf_pipe, rfparams, cv=5, n_jobs=-1)

In [105]:
rfgs.fit(eleven_train_df, madelon_train_targets[0])

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=350, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'clf__max_features': ['auto', 'log2'], 'clf__criterion': ['gini', 'entropy'], 'clf__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [106]:
rfgs.best_params_

{'clf__criterion': 'entropy',
 'clf__max_depth': None,
 'clf__max_features': 'auto'}

In [107]:
rfgs.score(eleven_train_df, madelon_train_targets[0])

1.0

In [108]:
rfgs.score(eleven_val_df,madelon_val_target[0])

0.87

In [121]:
eleven_train_df['targets'] = madelon_train_targets
eleven_val_df['targets'] = madelon_val_target

,48,64,105,128,241,336,338,378,442,453,475,targets
0,440,648,181,452,434,658,628,419,568,471,401,-1
1,499,488,431,473,551,469,528,526,463,311,549,-1
2,460,485,593,487,474,465,431,464,503,606,454,-1
3,529,415,698,493,569,398,377,553,447,545,602,1
4,429,387,451,475,538,385,509,424,536,426,560,1
5,464,348,667,493,545,316,429,468,445,428,557,1
6,447,496,571,485,418,476,538,444,578,657,389,1
7,496,384,584,484,590,372,437,516,401,388,609,-1
8,524,421,700,497,535,396,355,532,432,597,541,1
9,557,628,580,489,464,640,407,590,351,543,433,-1


In [122]:
# computes the train score and test score for both raw and scaled data for this particular model (with BaggingClassifier) using only 10% of the dataset samples 3 times.
# 200 train observations, 60 validation observations sampled 3 separate times

eleven_rf_rfe = add_train_test_score_df(eleven_train_df, eleven_val_df, RandomForestClassifier(criterion = 'entropy',
                                                                                  max_depth = None,
                                                                                  max_features = 'auto',
                                                                                  n_estimators = 350,
                                                                                random_state = 42),
                                       feats = 11)
eleven_rf_rfe_scaled = add_train_test_score_df(eleven_train_df, eleven_val_df, 
                                               RandomForestClassifier(criterion = 'entropy', 
                                                                      max_depth = None, 
                                                                      max_features = 'auto', 
                                                                      n_estimators = 350, 
                                                                      random_state = 42), 
                                                scaled = True,
                                                feats = 11)
eleven_rf_rfe, eleven_rf_rfe_scaled

([1.0, 0.68333333333333324], [1.0, 0.67222222222222217])